Task fnc-1 - classifying using merged two tf-idf representations

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from scipy.sparse import hstack
from sklearn.model_selection import cross_val_score

In [2]:
train_bodies = pd.read_csv('./data/train_bodies.csv')
train_stancies = pd.read_csv('./data/train_stances.csv')
test_bodies = pd.read_csv('./data/competition_test_bodies.csv')
test_stancies = pd.read_csv('./data/competition_test_stances.csv')

Merge two datasets into one

In [3]:
train = pd.merge(left=train_bodies, right=train_stancies, left_on='Body ID', right_on='Body ID')

In [4]:
test = pd.merge(left=test_bodies, right=test_stancies, left_on='Body ID', right_on='Body ID')

In [5]:
train

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated
...,...,...,...,...
49967,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets tipped more than $2,00...",agree
49968,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets $2,000 tip",agree
49969,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",agree
49970,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...",Ann Arbor pizza delivery driver surprised with...,agree


In [6]:
train = train.drop_duplicates()
train

,Body ID,articleBody,Headline,Stance
0,0,A small meteorite crashed into a wooded area i...,"Soldier shot, Parliament locked down after gun...",unrelated
1,0,A small meteorite crashed into a wooded area i...,Tourist dubbed ‘Spider Man’ after spider burro...,unrelated
2,0,A small meteorite crashed into a wooded area i...,Luke Somers 'killed in failed rescue attempt i...,unrelated
3,0,A small meteorite crashed into a wooded area i...,BREAKING: Soldier shot at War Memorial in Ottawa,unrelated
4,0,A small meteorite crashed into a wooded area i...,Giant 8ft 9in catfish weighing 19 stone caught...,unrelated
...,...,...,...,...
49966,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery driver surprised with $2,000 tip",agree
49967,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets tipped more than $2,00...",agree
49968,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Pizza delivery man gets $2,000 tip",agree
49969,2532,"ANN ARBOR, Mich. – A pizza delivery man in Mic...","Luckiest Pizza Delivery Guy Ever Gets $2,000 Tip",agree


In [7]:
test

,Body ID,articleBody,Headline,Stance
0,1,Al-Sisi has denied Israeli reports stating tha...,Apple installing safes in-store to protect gol...,unrelated
1,1,Al-Sisi has denied Israeli reports stating tha...,El-Sisi denies claims he'll give Sinai land to...,agree
2,1,Al-Sisi has denied Israeli reports stating tha...,Apple to keep gold Watch Editions in special i...,unrelated
3,1,Al-Sisi has denied Israeli reports stating tha...,Apple Stores to Keep Gold “Edition” Apple Watc...,unrelated
4,1,Al-Sisi has denied Israeli reports stating tha...,South Korean woman's hair 'eaten' by robot vac...,unrelated
...,...,...,...,...
25408,2586,Remember how much Republicans wanted to repeal...,A Sign That Obamacare Exchanges Are Failing,disagree
25409,2586,Remember how much Republicans wanted to repeal...,Republicans call Obamacare a 'failure.' These ...,agree
25410,2586,Remember how much Republicans wanted to repeal...,CBO’s Alternate Facts Show Obamacare is Unsust...,disagree
25411,2586,Remember how much Republicans wanted to repeal...,Why Obamacare failed,disagree


In [8]:
encoder = LabelEncoder().fit(train['Stance'])

In [9]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
        
    def __call__(self, text):
        return [self.wnl.lemmatize(token) for token in word_tokenize(text)]

In [10]:
X_train_headline, X_train_text, y_train = train['Headline'], train['articleBody'], encoder.transform(train['Stance'])

In [11]:
X_test_headline, X_test_text, y_test = test['Headline'], test['articleBody'], encoder.transform(test['Stance'])

In [12]:
collected_train_texts = train.apply(lambda x: x['articleBody'] + ' ' + x['Headline'], axis=1)

In [13]:
vec = CountVectorizer(tokenizer=LemmaTokenizer()).fit(collected_train_texts)

/home/ubuntu/.local/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:489: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


In [14]:
X_train_headline_counts = vec.transform(X_train_headline)
X_train_text_counts = vec.transform(X_train_text)

In [15]:
X_test_headline_counts = vec.transform(X_test_headline)
X_test_text_counts = vec.transform(X_test_text)

In [16]:
X_train_diff_counts = X_train_text_counts - X_train_headline_counts

In [17]:
X_test_diff_counts = X_test_text_counts - X_test_headline_counts

Select best attributes for classifiers - chi2 with pvalue < 0.05

GradientBoosting classifier for differenced word counts (text - headline)

In [18]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()
cross_val_score(classifier, X_train_diff_counts, y_train, cv=5, n_jobs=5)

array([0.72120234, 0.66108533, 0.66542263, 0.65079685, 0.70990518])

SVC classifier for differenced word counts (text - headline)

In [19]:
from sklearn.svm import SVC

classifier = SVC()
cross_val_score(classifier, X_train_diff_counts, y_train, cv=5, n_jobs=5)

array([0.70254186, 0.44785152, 0.39358483, 0.37835384, 0.58674602])

Gradient boosting classifier has better result - check final all test result

In [22]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier().fit(X_train_diff_counts, y_train)
print(classifier.score(X_test_diff_counts, y_test))

0.707590603234565
